# Imports

In [1]:
import pandas as pd 
import numpy as np
from datetime import datetime, timedelta
import geopandas
import matplotlib.pyplot as plt
import matplotlib.animation
from matplotlib.animation import FuncAnimation
import mapclassify
import matplotlib.patches as mpatches

import sys
sys.path.insert(0, "/Users/jennavergeynst/OneDrive - UGent/Corona_Postdoc/Analyses/tools")
from plotting import add_scalebar, add_north_arrow

pip install celluloid

# Prepare community shapes

Origin shapefile:

https://ac.ngi.be/remoteclient-open/ngi-standard-open/Vectordata/AdminVector/AdminVector_L08_shp.zip


In [ ]:
communities_geom = geopandas.read_file('../data/NGI_AdminVector_L08_shp/AD_2_Municipality.shp')
NGI_url = 'zip+https://ac.ngi.be/remoteclient-open/ngi-standard-open/Vectordata/AdminVector/AdminVector_L08_shp.zip'
communities_geom.NISCode = communities_geom.NISCode.astype(int)
communities_geom = communities_geom[['NISCode','geometry']]

In [ ]:
# read inhabitants
url_inhabitants = 'https://statbel.fgov.be/sites/default/files/files/documents/bevolking/5.1%20Structuur%20van%20de%20bevolking/Bevolking_per_gemeente.xlsx'
inhabitants = pd.read_excel(url_inhabitants, skipfooter=4, skiprows=1).dropna(how='all', axis=0)
inhabitants['NIS code'] = inhabitants['NIS code'].astype(int)


In [ ]:
## Uncomment to save locally:
#inhabitants.to_csv('../data/inhabitants.csv')
## Uncomment to read locally:
#inhabitants = pd.read_csv('../data/inhabitants.csv')

In [ ]:
communities = pd.merge(communities_geom, inhabitants, left_on = 'NISCode', right_on = 'NIS code').drop(columns=['NIS code'])
communities = communities.rename(columns={'Totaal':'inhabitants'})

In [ ]:
communities.plot('inhabitants')

# Read sciensano data

In [ ]:
## To update data until yesterday
#datelist = pd.date_range(start = pd.to_datetime('2020-03-31'), end = datetime.today() - timedelta(days=1)).strftime("%Y%m%d")
datelist = pd.date_range(start = pd.to_datetime('2020-03-31'), end = pd.to_datetime('2020-05-11')).strftime("%Y%m%d")

In [ ]:
datelist = list(datelist) + [datelist[-1]]*20

In [ ]:
## Safe data locally (to be safe)
# for date in datelist:
#     url = 'https://epistat.sciensano.be/Data/'+ date +'/COVID19BE_CASES_MUNI_CUM_'+ date +'.csv'
#     cases_day_x = pd.read_csv(url, encoding = "ISO-8859-1", skipfooter=1).dropna(axis=0, how='all').copy()
#     cases_day_x.to_csv('../data/cumulative_cases_per_community/'+'COVID19BE_CASES_MUNI_CUM_'+ date +'.csv')

In [ ]:
def prepare_case_data(date, read_local = True):
    if read_local == True:
        cases_day_x = pd.read_csv('../data/cumulative_cases_per_community/'+'COVID19BE_CASES_MUNI_CUM_'+ date +'.csv')
    else:
        url = 'https://epistat.sciensano.be/Data/'+ date +'/COVID19BE_CASES_MUNI_CUM_'+ date +'.csv'
        cases_day_x = pd.read_csv(url, encoding = "ISO-8859-1", skipfooter=1).dropna(axis=0, how='all').copy()
    cases_day_x.NIS5 = cases_day_x.NIS5.astype(int)
    cases_day_x = cases_day_x[['NIS5','TX_DESCR_NL','CASES']].copy()
    # Assume <5 cases to be 3 cases
    cases_day_x.loc[cases_day_x.CASES.str.find('<')!=-1, 'CASES'] = '3'
    cases_day_x.CASES = cases_day_x.CASES.astype(int)
    return cases_day_x

In [ ]:
def prepare_plot_data(date):
    cases_day_x = prepare_case_data(date)
    cases_with_geo = pd.merge(communities[['NISCode', 'geometry', 'inhabitants']] , cases_day_x[['NIS5','CASES']], left_on = 'NISCode', right_on = 'NIS5')
    cases_with_geo['cases_per_100000'] = cases_with_geo.CASES/cases_with_geo.inhabitants*100000
    return cases_with_geo

# Make map

https://geopandas.org/mapping.html

mapclassify:
https://nbviewer.jupyter.org/github/pysal/mapclassify/blob/master/notebooks/south.ipynb

time formats:
https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes

interactive animation:
https://nbviewer.jupyter.org/github/jakevdp/JSAnimation/blob/master/animation_example.ipynb

**For now: just a dynamic GIF!**

In [ ]:
cases_with_geo = prepare_plot_data(datelist[0])

**fisher_jenks scheme**: seeks to reduce the variance within classes and maximize the variance between classes (This is done by seeking to minimize each class's average deviation from the class mean, while maximizing each class's deviation from the means of the other groups.)

http://darribas.org/gds15/content/labs/lab_04.html

In [ ]:
last = prepare_plot_data(datelist[-2])
classi = mapclassify.FisherJenks(last['cases_per_100000'], k=7)
FJ_bins = classi.bins.round(-1).astype(int)
FJ_bins = [0]+list(FJ_bins)
FJ_bins[-1] = 100000

In [ ]:
pd.cut(last['cases_per_100000'], bins= FJ_bins).value_counts()

In [ ]:
# Prepare labels
lower_lims = [str(x) for x in FJ_bins[:-1]]
lower_lims[-1] = '> '+lower_lims[-1]
upper_lims = [' - ' + str(x) for x in FJ_bins[1:]]
upper_lims[-1] = ''
labels = [x+y for x,y in zip(lower_lims, upper_lims)]

In [ ]:
# union belgium 
Belgium = last.unary_union

In [ ]:
fig, ax = plt.subplots(dpi=150, figsize=(8,8))

last.plot(column=pd.cut(last['cases_per_100000'], bins= FJ_bins), cmap='Oranges', 
          legend=True, ax=ax, vmin=-1, vmax=6)
leg = ax.get_legend()
patches = [mpatches.Patch(color=p.get_markerfacecolor()) for p in leg.legendHandles]
ax.legend(patches, labels, loc = (0, 0.1), frameon=False, bbox_transform=ax.transAxes, title='# infected / 100000')


In [ ]:
def cleaned_plot(date, ax):
    ax.set_axis_off()

    temp = prepare_plot_data(date)
    temp.plot(column=pd.cut(temp['cases_per_100000'], bins= FJ_bins), cmap='Oranges', 
              legend=False, ax=ax, vmin=-1, vmax=6)
    ax.legend(patches, labels, loc = (0, 0.1), frameon=False, bbox_transform=ax.transAxes, title='# infected / 100000')

    geopandas.GeoSeries([Belgium.boundary]).plot(ax=ax, color='k', lw=0.1)

    add_north_arrow(ax, (0.9, 0.1))
    add_scalebar(ax, (512000, 510030), length=50000, fac=' 50 km')
    
    ax.set_title('Cumulative cases per municipality on ' + pd.to_datetime(date).strftime('%d %B %Y'))
    
    return ax

In [ ]:
def anim_func(frame, *fargs):
    ax,  = fargs
    extent = ax.axis()
    ax.clear()
    ax = cleaned_plot(frame, ax)
    ax.axis(extent)
    ax.set_axis_off()
    ax.get_figure().tight_layout()
    return ax

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
ax = cleaned_plot(datelist[0], ax)
fig.tight_layout()
anim = matplotlib.animation.FuncAnimation(fig, anim_func, frames=datelist, fargs=(ax, ), interval=250)

In [ ]:
anim.save("../results/map_cases_dynamic.gif", writer='imagemagick', savefig_kwargs=dict(bbox_inches='tight'))#, savefig_kwargs=dict(bbox='tight'))

In [ ]:
#anim.to_html5_video()

# Html example

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.resources import INLINE
output_notebook(resources=INLINE)

In [ ]:
from bokeh.plotting import figure
from bokeh.resources import CDN
from bokeh.embed import file_html

plot = figure()
plot.circle([1,2], [3,4])

html = file_html(plot, CDN, "my plot")


In [ ]:
show(plot)

In [ ]:
print(html)